In [1]:
# from ufal.udpipe import Model as udModel, Pipeline
import re
import alternation

In [2]:
# model_path = "finnish-tdt-ud-2.5-191206.udpipe"
# ud_model = udModel.load(model_path)

In [2]:
def star_and_vocals(word):
    token = word[::-1]
    vocals = ['e', 'y', 'u', 'i', 'o', 'a', 'ä', 'ö']
    for i in range(len(token)):
        if token[i] not in vocals:
            token = token[:i] + '*' + token[i:]
            token = token[::-1]
            break
    return token

In [13]:
def process_psor(token, person, *, number={'Sing'}):
    """
    Erases possesive suffix
    token - string, word form
    number - set which contains string 'Sing' or 'Plur'
    person - set which contains string
    """
    output = token
    if '1' in person:
        if 'Sing' in number:
            output = re.sub(r"ni$", "", token)
        elif 'Plur' in number:
            output = re.sub(r"mme$", "", token)
    elif '2' in person:
        if 'Sing' in number:
            output = re.sub(r"si$", "", token)
        elif 'Plur' in number:
            output = re.sub(r"nne$", "", token)
    elif '3' in person:
        output = re.sub(r"((ns[aä])|([aeiouäöy]n))$", "", token)  
    return output
            

def process_case(token, case, number={'Sing'}):
    """
    deletes case marker
    token - string, word form
    case - set which contains string
    number - set which contains string 'Sing' or 'Plur'
    """
    output = token
    if 'Abl' in case:
        output = re.sub(r"lt[aä]$", "", token)
    elif 'Ade' in case:
        output = re.sub(r"ll[aä]$", "", token)
    elif 'All' in case:
        output = re.sub(r"lle$", "", token)
    elif 'Ela' in case:
        output = re.sub(r"st[aä]$", "", token)
    elif 'Ess' in case:
        output = re.sub(r"n[aä]$", "", token)
    elif 'Gen' in case:
        if 'Sing' in number:
            output = re.sub(r"n?$", "", token)
        elif 'Plur' in number:
            output = re.sub(r"(tt|d|t)?en?$", "", token)
    elif 'Ill' in case:
        output = re.sub(r"(?<=ee)seen?$", "", token)
        output = re.sub(r"h?[aeiouäöy]n?$", "", output)
    elif 'Ine' in case:
        output = re.sub(r"ss[aä]$", "", token)
    elif 'Par' in case:
        output = re.sub(r"(t?[aä])$", "", token)
    elif 'Tra' in case:
        output = re.sub(r"ksi$", "", token)
    elif 'Nom' in case and 'Plur' in number:
        output = re.sub(r"t?$", "", token)
        output = star_and_vocals(output)[::-1]
        counter = 0
        for i in range(len(output)):
            if output[i] != '*':
                counter += 1
            else:
                break
        if counter == 1:
            output = alternation.alternation_backward(output[::-1])
        else:
            output = re.sub(r"\*", "", output)
            output = output[::-1]
    return output

In [10]:
def process_number(word, case, number={'Sing'}):
    output = word
    if len(output) < 2:
        return output
    if 'Sing' in number or ('Nom' in case and 'Plur' in number):
        if 'Nom' in case and 'Sing' in number:
            return output
        
        if 'Par' in case:
            output = re.sub(r"(?<=i)s$", "nen", output)
            if output != word:
                return output
            output = re.sub(r"([hlrn])$", r"\1i", output)
            output = re.sub(r"([a-zäö]{2,}[uy])t$", r"\1s", output)
            output = re.sub(r"e$", r"i", output)
            output = re.sub(r"([a-zäö]{2,}e)t$", r"\1", output)
            output = re.sub(r"t$", "si", output)
            return output
        
        
        # исключения    
        output = re.sub(r"^lapse$", "lapsi", output)
        output = re.sub(r"^miehe$", "mies", output)
        
        # блок основ с обратными чередованиями при образовании местных падежей
        output = re.sub(r"ttoma$", "ton", output)
        output = re.sub(r"ttömä$", "tön", output)
        output = re.sub(r"(e[nlr]|[aä]r)e$", r"*\1", output)
#         output = re.sub(r"ttar$", "tar", output)
#         output = re.sub(r"ttär$", "tär", output)
        output = re.sub(r"ime$", r"*in", output)
        output = re.sub(r"ii$", r"*is", output)
        output = re.sub(r"([a-z]{3,})aa$", r"\1*as", output)
        output = re.sub(r"([a-zäö]{3,})ää$", r"\1*äs", output)
        # магия сингармонизма гласных
        output = re.sub(r"([qwrtuopasdfghjklzxcvbnm]{3,})nee$", r"\1nut", output)
        output = re.sub(r"([a-zäö]{3,})nee", r"\1nyt", output)
        output = re.sub(r"ee$", r"*e", output)
        output = alternation.alternation_forward(output)
        if output != word:
            return output
        
        # разные типы на -s
        output = re.sub(r"kse$", "s", output)
        output = re.sub(r"([a-zäö]{2,}[aeiouyäö])de$", r"\1s", output)
        output = re.sub(r"([a-zäö]{2,}[aeiouyäö])te$", r"\1s", output) # Mihin
        if output != word:
            return output
        
        output = re.sub(r"([a-zäö]{2,}u|y)e$", r"\1t", output)
        output = re.sub(r"se$", "nen", output)
        output = re.sub(r"de$", "si", output)
        output = re.sub(r"te$", "si", output)
        if output != word:
            return output
        
        if 'Nom' not in case and 'Par' not in case and 'Ill' not in case and 'Ess' not in case:
            output = re.sub(r"([a-zäö]{2,})([aeiouyäö])$", r"\1*\2", output)
            output = alternation.alternation_backward(output)
        output = re.sub(r"e$", "i", output)
    
    elif 'Plur' in number and 'Nom' not in case:
 
        #  с генитивом беда - надо отдельно разобрать
 
        # other cases
        output = re.sub(r"^planeetto(i$|j$)", "planeetta", output) #работает
        output = re.sub(r"^(lapsi|laps)$", "lapsi", output) #работает
        output = re.sub(r"^miehi$", "mies", output) #работает
 
        # нормальные ребята, которые точно могут быть получены из мн. ч.
        # ребята с -ei- or -ej-
        output = re.sub(r"(?<=un)(ei|ej)$", "ut", output)
        output = re.sub(r"(?<=yn)(ei|ej)$", "yt", output)
 
        output = re.sub(r"(?<=(tt|lt|rk|mp|nk|kk|nt|pp|rt))(ei|ej)$", "*e", output)
        output = re.sub(r"(?<=(t|p|k))(ei|ej)$", "*e", output)
        output = alternation.alternation_forward(output)
        if output != word:
            return output
            # заимствованные слова на -i- - не знаю, что делвть с их чередованиями, смотри об этом ниже
        output = re.sub(r"(ei|ej)$", "i", output)
 
        output = re.sub(r"(?<=i)si$", "nen", output)
        # слова типа uusi отбросом окончанния тут готовы
        output = re.sub(r"ksi$", "s", output) #работает
 
 
        output = re.sub(r"([a-zäö]{3,})imi$", r"\1*in", output) #работает
        output = alternation.alternation_forward(output)
        if output != word:
            return output
        # слова типа kieli pieni etc. автоматом получаются отсечением падежа
        output = re.sub(r"ttomi$", "ton", output)
        output = re.sub(r"ttömi$", "tön", output)
        output = re.sub(r"ttari$", "tar", output)
        output = re.sub(r"ttäri$", "tän", output)
        output = re.sub(r"(?<=(ly|vy|hy|lu|vu|hu))i$", r"t", output)
        if output != word:
            return output
        if len(output) >= 7:
            output = re.sub(r"(?<=(l|n|r))(oj|oi)$", "a", output) #работает
            output = re.sub(r"(?<=(l|n|r))(öj|öi)$", "ä", output) #работает
            output = re.sub(r"(?<=k)(oj|oi)$", "*a", output)  #работает
            output = re.sub(r"(?<=k)(öj|öi)$", "*ä", output)  #работает
            output = alternation.alternation_backward(output)
            if output != word:
                return output
 
 
        output = re.sub(r"(?<=(ij|kk))(oj|oi)$", "a", output)  # работает
        output = re.sub(r"(?<=(ij|kk))(öj|öi)$", "ä", output)  # работает
 
        output = re.sub(r"([a-z]{3,})ai$", r"\1*as", output) #работает
        output = re.sub(r"([a-zäö]{3,})äi$", r"\1*äs", output) #работает
        output = alternation.alternation_forward(output)
        if output != word:
            return output
 
        output = re.sub(r"ii$", r"is", output)
 
        output = re.sub(r"([a-z]{2,})o(j|i)$", r"\1a", output)
        output = re.sub(r"([a-zäö]{2,})ö(j|i)$", r"\1ä", output)
 
        #output = re.sub(r"([a-z]{4,})i$", r"\1a", output)
        #output = re.sub(r"([a-zäö]{4,})i$", r"\1ä", output)
        if len(output) == 3:
            output = re.sub(r"äi$", r"ää", output)
            output = re.sub(r"ai$", r"aa", output)
            output = re.sub(r"ui$", r"uu", output)
            output = re.sub(r"yi$", r"yy", output)
 
        # все остальные случаи с сильными гласными
        output = re.sub(r"(?<![qwrtplkjhgfdszxcvbnms])(i|j)$", r"", output)
 
        output = re.sub(r"s$", r"nen", output)
 
        if 'Nom' not in case and 'Par' not in case and 'Ill' not in case and 'Ess' not in case:
            if len(output) > 3:
                output = star_and_vocals(output)
                output = alternation.alternation_backward(output)
                if output != word:
                    return output
 
        
    return output

In [11]:
def process_token(token):
    output = token.form
    output = output.replace('-', '').replace(':', '')
    if len(output) > 4 and output.isupper():
        output = output.lower()
    if 'Clitic' in token.feats:
        output = re.sub(r"kin$", "", output)
    if 'Person[psor]' in token.feats:
        if 'Number[psor]' in token.feats:
            output = process_psor(output, token.feats['Person[psor]'], number=token.feats['Number[psor]'])
        else:
            output = process_psor(output, person=token.feats['Person[psor]'])
    if 'Case' in token.feats:
        case = token.feats['Case']
        if 'Number' in token.feats:
            number = token.feats['Number']
            output = process_case(output, case, number)
            output = process_number(output, case, number)
        else:
            output = process_case(output, case)
            output = process_number(output, case)
    return output

### Тест модели на заведомо корректных данных

In [17]:
import pyconll

sents = pyconll.load_from_file("fi_tdt-ud-test.conllu")
answer = []
error = []
nouns_all_sing = 0
nouns_all_plur = 0
nouns_correct_sing = 0
nouns_correct_plur = 0
nouns_other = 0
nouns_correct_other = 0
for sentence in sents:
    for token in sentence:
        if token.xpos in ['A', 'N']:
            if 'Number' in token.feats:
                processed = process_token(token)
                correct_ending = token.lemma.replace('#', '')
                if 'Sing' in token.feats['Number']:
                    nouns_all_sing += 1
                    if processed.lower() == correct_ending.lower():
                        nouns_correct_sing += 1

                elif 'Plur' in token.feats['Number']:
                    nouns_all_plur += 1
                    if processed.lower() == correct_ending.lower():
                        nouns_correct_plur += 1



                if processed.lower() == correct_ending.lower():
                    answer.append((token.form, processed))
                else:
                    error.append((token.form, correct_ending, processed))

total_nouns = nouns_all_sing + nouns_all_plur
total_correct = nouns_correct_sing + nouns_correct_plur
print("Sing: {0} out of {1}, accuracy = {2}".format(nouns_correct_sing, nouns_all_sing,
                                                    nouns_correct_sing / nouns_all_sing))
print("Plur: {0} out of {1}, accuracy = {2}".format(nouns_correct_plur, nouns_all_plur,
                                                    nouns_correct_plur / nouns_all_plur))
print("Total: {0} out of {1}, accuracy = {2}".format(total_correct, total_nouns,
                                                     total_correct / total_nouns))

Sing: 5184 out of 6233, accuracy = 0.8317022300657789
Plur: 1124 out of 1926, accuracy = 0.5835929387331257
Total: 6308 out of 8159, accuracy = 0.7731339624954039


In [18]:
error[:100]

[('pupulle', 'pupu', 'puppu'),
 ('Suxessiin', 'Suxess', 'Suxessi'),
 ('tupaantuliaiset', 'tupaantuliaiset', 'tupaantuliainen'),
 ('mukavat', 'mukava', 'mukapa'),
 ('pupun', 'pupu', 'puppu'),
 ('mukaviin', 'mukava', 'mukavi'),
 ('Mielestäni', 'mieli', 'Miel'),
 ('juhlien', 'juhla', 'juhli'),
 ('hauskin', 'hauska', 'hauskin'),
 ('ystävien', 'ystävä', 'ystäpi'),
 ('häissä', 'häät', 'hää'),
 ('hautajaisissa', 'hautajaiset', 'hautajainen'),
 ('Tupaantuliaisista', 'tupaantuliaiset', 'Tupaantuliainen'),
 ('Suxessiin', 'Suxess', 'Suxessi'),
 ('mieheni', 'mies', 'miehe'),
 ('tyttönimeni', 'tyttönimi', 'tyttönime'),
 ('päässäni', 'pää', 'päkä'),
 ('ammatteja', 'ammatti', 'ammate'),
 ('pukuliivistä', 'pukuliivi', 'pukuliipi'),
 ('nuorenmiehen', 'nuorimies', 'nuorenmiehi'),
 ('mielessäni', 'mieli', 'miel'),
 ('pidempi', 'pitkä', 'pidempi'),
 ('ystäville', 'ystävä', 'ystäpi'),
 ('puhtain', 'puhdas', 'puhtain'),
 ('sydämin', 'sydän', 'sydämin'),
 ('suhdeongelmiin', 'suhdeongelma', 'suhdeongelmi'),
 